In [26]:
import os
os.environ['MKL_NUM_THREADS'] = '1'
import argparse
import codecs
import logging
import time
import tqdm
import pandas as pd

import implicit.cuda

import numpy as np

from scipy.sparse import load_npz
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
#from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
#                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
from implicit.datasets.lastfm import get_lastfm
from implicit.datasets.million_song_dataset import get_msd_taste_profile

from implicit.evaluation import train_test_split
from implicit.evaluation import mean_average_precision_at_k

class RandomRecommender:
    def __init__(self):
#        self.min_bound = min_bound
#        self.high_bound = high_bound
#        self.confidence = confidence
        self.something=1
    
    def recommend(self,userid,train_csr,N):
        recs = np.random.randint(self.min_bound,self.high_bound,N)
        recs = [(rec,self.confidence) for rec in recs]
        return recs
    
    def fit(self,train_data):
        self.min_bound = 0
        self.high_bound = train_data.shape[0]
        self.confidence = 1
        return 1

# train_plays, test_plays -> num_songs x num_users CSR matrix
# train_plays = load_npz('data/train_sparse.npz')
# test_plays = load_npz('data/test_sparse.npz')
# # songs -> CSR_row_index: song_id
# songs = pd.read_hdf('data/song_mapping.h5', key='df')
# # songs -> CSR_col_index: user_id
# users = pd.read_hdf('data/user_mapping.h5', key='df')

def build_cf_model(train_plays):
    als_params = {'factors': 16,
                  'dtype': np.float32,
                  'iterations': 15,
                  'calculate_training_loss': True}
    model = AlternatingLeastSquares(**als_params)
    #perform operation to approximate confidence intervals
    #paper doesn't specify an alpha value, so just guess alpha=1
    alpha = 1
    train_plays.data = 1 + np.log(alpha*train_plays.data)
    test_plays.data = 1 + np.log(alpha*test_plays.data)
    model.fit(train_plays)
    return model

def get_recommendations(model, user_ids, train_plays)
    recs = {}
    for user_id in user_ids:
        recs[user_id] = model.recommend(user_id, train_plays.transpose())

# TODO: user_ids is passed to this file


,sparse_index,track
0,221367,SOOFKYO12AF72A2640
1,132143,SOIHOIQ12A8C138593
2,363912,SOYIZSN12A6701E0BB
3,63740,SODYZAD12A58A7A525
4,351788,SOXLWPN12A8C143667


In [18]:
test_plays.indices

array([ 714415,   49664,  796098, ..., 1112360,  361288,  786004],
      dtype=int32)

In [19]:
# songs = pd.read_hdf('data/song_mapping.h5', key='df')
# users = pd.read_hdf('data/user_mapping.h5', key='df')


,track,sparse_index
0,221367,SOOFKYO12AF72A2640
1,132143,SOIHOIQ12A8C138593
2,363912,SOYIZSN12A6701E0BB
3,63740,SODYZAD12A58A7A525
4,351788,SOXLWPN12A8C143667


In [25]:
# songs = songs.rename(index=str, columns={'track': 'sparse_index', 'sparse_index': 'track'})
# songs.head()
# songs.to_hdf('data/song_mapping.h5', key='df')

# users = users.rename(index=str, columns={'user': 'sparse_index', 'sparse_index': 'user'})
# users.head(100)
# users.to_hdf('data/user_mapping.h5', key='df')